In [4]:
import os
from config import config
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY


## **<font color="blue">InMemorySessionService</font>**

In [1]:
import asyncio
from google.adk.agents import LlmAgent
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.tools import load_memory
from google.genai.types import Content, Part
import os
from config import config
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY


# --- Constants ---
APP_NAME = "app_01"
USER_ID = "user_11"
MODEL = "gemini-2.5-flash"


# --- Agent 1: Simple agent to capture information ---
info_capture_agent = LlmAgent(
    model=MODEL,
    name="InfoCaptureAgent",
    instruction="Acknowledge the user's statement.",
)


# --- Agent 2: Agent that can use memory ---
memory_recall_agent = LlmAgent(
    model=MODEL,
    name="MemoryRecallAgent",
    instruction=(
        "Answer the user's question. "
        "Use the 'load_memory' tool if the answer might be in past conversations."
    ),
    tools=[load_memory]
)


# --- Shared Services ---
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()


async def run_scenario():

    # ==============================
    # Turn 1: Capture Information
    # ==============================
    print("\n--- Turn 1: Capturing Information ---")

    runner1 = Runner(
        agent=info_capture_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session1_id = "session_info"

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    user_input1 = Content(
        role="user",
        parts=[Part(text="My favorite project is Project Alpha.")]
    )

    final_response_text = "(No final response)"

    async for event in runner1.run_async(
        user_id=USER_ID,
        session_id=session1_id,
        new_message=user_input1
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text = event.content.parts[0].text

    print(f"Agent 1 Response: {final_response_text}")

    # Get completed session
    completed_session1 = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    # Add to memory
    print("\n--- Adding Session 1 to Memory ---")
    await memory_service.add_session_to_memory(completed_session1)
    print("Session added to memory")


    # ==============================
    # Turn 2: Recall Information
    # ==============================
    print("\n--- Turn 2: Recalling Information ---")

    runner2 = Runner(
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session2_id = "session_recall"

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session2_id
    )

    user_input2 = Content(
        role="user",
        parts=[Part(text="What is my favorite project?")]
    )

    final_response_text_2 = "(No final response)"

    async for event in runner2.run_async(
        user_id=USER_ID,
        session_id=session2_id,
        new_message=user_input2
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text_2 = event.content.parts[0].text

    print(f"Agent 2 Response: {final_response_text_2}")


# Proper Python entry point
if __name__ == "__main__":
    # asyncio.run(run_scenario())
    await run_scenario()



--- Turn 1: Capturing Information ---
Agent 1 Response: Acknowledged. Your favorite project is Project Alpha.

--- Adding Session 1 to Memory ---
Session added to memory

--- Turn 2: Recalling Information ---


D:\Agent-Development-Kit\venv\Lib\site-packages\google\adk\flows\llm_flows\base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Agent 2 Response: Your favorite project is Project Alpha.


## **<font color="blue">DatabaseSessionService</font>**

In [2]:
import os
import asyncio
from config import config

from google.adk.agents import LlmAgent
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.tools import load_memory
from google.genai.types import Content, Part


# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "app_01"
USER_ID = "user_11"
MODEL = "gemini-2.5-flash"

# SQLite async DB
db_url = "sqlite+aiosqlite:///./db_agent_data.db"


# -------------------------------------------------
# Agent Definitions
# -------------------------------------------------

info_capture_agent = LlmAgent(
    model=MODEL,
    name="InfoCaptureAgent",
    instruction="Acknowledge the user's statement.",
)

memory_recall_agent = LlmAgent(
    model=MODEL,
    name="MemoryRecallAgent",
    instruction=(
        "Answer the user's question. "
        "Use the 'load_memory' tool if the answer might be in past conversations."
    ),
    tools=[load_memory]
)


# -------------------------------------------------
# Shared Services (Persistent Sessions)
# -------------------------------------------------

session_service = DatabaseSessionService(db_url=db_url)
memory_service = InMemoryMemoryService()  # Still RAM-based


async def run_scenario():

    # Initialize DB tables (safe across versions)
    if hasattr(session_service, "initialize"):
        await session_service.initialize()

    # =============================================
    # Turn 1: Capture Information
    # =============================================
    print("\n--- Turn 1: Capturing Information ---")

    runner1 = Runner(
        agent=info_capture_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session1_id = "session_info"

    # Create or ensure session exists
    existing_session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    if not existing_session:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session1_id
        )

    user_input1 = Content(
        role="user",
        parts=[Part(text="My favorite project is Project Alpha.")]
    )

    final_response_text = "(No final response)"

    async for event in runner1.run_async(
        user_id=USER_ID,
        session_id=session1_id,
        new_message=user_input1
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text = event.content.parts[0].text

    print(f"Agent 1 Response: {final_response_text}")

    # Retrieve completed session from DB
    completed_session1 = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    print("\n--- Adding Session 1 to Memory ---")
    await memory_service.add_session_to_memory(completed_session1)
    print("Session added to memory")


    # =============================================
    # Turn 2: Recall Information (New Session)
    # =============================================
    print("\n--- Turn 2: Recalling Information ---")

    runner2 = Runner(
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session2_id = "session_recall"

    existing_session2 = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session2_id
    )

    if not existing_session2:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session2_id
        )

    user_input2 = Content(
        role="user",
        parts=[Part(text="What is my favorite project?")]
    )

    final_response_text_2 = "(No final response)"

    async for event in runner2.run_async(
        user_id=USER_ID,
        session_id=session2_id,
        new_message=user_input2
    ):
        if event.is_final_response() and event.content and event.content.parts:
            final_response_text_2 = event.content.parts[0].text

    print(f"Agent 2 Response: {final_response_text_2}")


# -------------------------------------------------
# Entry Point
# -------------------------------------------------
if __name__ == "__main__":
    # asyncio.run(run_scenario())
    await run_scenario()


Event from an unknown agent: MemoryRecallAgent, event id: 5e8ad763-3ba9-4516-9816-bb01a6734419
Event from an unknown agent: MemoryRecallAgent, event id: e7fd80b4-77c2-4e22-965b-f6dbc0e7d51a
Event from an unknown agent: MemoryRecallAgent, event id: 6cb4f9f0-9acb-4cad-807d-488f5e4a99e2



--- Turn 1: Capturing Information ---
Agent 1 Response: You told me: My favorite project is Project Alpha.My favorite project is Project Alpha.

--- Adding Session 1 to Memory ---
Session added to memory

--- Turn 2: Recalling Information ---
Agent 2 Response: Your favorite project is Project Alpha.


In [3]:
import os
import asyncio
from config import config

from google.adk.agents import LlmAgent
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.tools import load_memory
from google.genai.types import Content, Part


# -------------------------------------------------
# Configuration
# -------------------------------------------------
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

APP_NAME = "app_01"
USER_ID = "user_11"
MODEL = "gemini-2.5-flash"

db_url = "sqlite+aiosqlite:///./db_agent_data.db"


# -------------------------------------------------
# Agents
# -------------------------------------------------

info_capture_agent = LlmAgent(
    model=MODEL,
    name="InfoCaptureAgent",
    instruction="Acknowledge the user's statement.",
)

memory_recall_agent = LlmAgent(
    model=MODEL,
    name="MemoryRecallAgent",
    instruction=(
        "Answer the user's question. "
        "Use the 'load_memory' tool if needed."
    ),
    tools=[load_memory]
)


# -------------------------------------------------
# Services
# -------------------------------------------------

session_service = DatabaseSessionService(db_url=db_url)
memory_service = InMemoryMemoryService()


async def run_scenario():

    # Initialize DB tables
    if hasattr(session_service, "initialize"):
        await session_service.initialize()

    # =============================================
    # Turn 1
    # =============================================
    print("\n--- Turn 1: Capturing Information ---")

    runner1 = Runner(
        agent=info_capture_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session1_id = "session_info"

    existing_session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    if not existing_session:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session1_id
        )

    user_input1 = Content(
        role="user",
        parts=[Part(text="My favorite project is Project Alpha.")]
    )

    async for _ in runner1.run_async(
        user_id=USER_ID,
        session_id=session1_id,
        new_message=user_input1
    ):
        pass

    completed_session1 = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session1_id
    )

    await memory_service.add_session_to_memory(completed_session1)
    print("Turn 1 completed & added to memory.")


    # =============================================
    # Turn 2
    # =============================================
    print("\n--- Turn 2: Recalling From Memory Tool ---")

    runner2 = Runner(
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    session2_id = "session_recall"

    existing_session2 = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session2_id
    )

    if not existing_session2:
        await session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=session2_id
        )

    user_input2 = Content(
        role="user",
        parts=[Part(text="What is my favorite project?")]
    )

    async for event in runner2.run_async(
        user_id=USER_ID,
        session_id=session2_id,
        new_message=user_input2
    ):
        if event.is_final_response() and event.content and event.content.parts:
            print("Agent 2 Response:", event.content.parts[0].text)


    # =============================================
    # Turn 3 (Load From SQLite Session History)
    # =============================================
    print("\n--- Turn 3: Loading From SQLite Session History ---")

    runner3 = Runner(
        agent=memory_recall_agent,
        app_name=APP_NAME,
        session_service=session_service,
        memory_service=memory_service
    )

    user_input3 = Content(
        role="user",
        parts=[Part(text="Repeat what I told you earlier.")]
    )

    async for event in runner3.run_async(
        user_id=USER_ID,
        session_id="session_info",
        new_message=user_input3
    ):
        if event.is_final_response() and event.content and event.content.parts:
            print("Agent 3 Response:", event.content.parts[0].text)


# -------------------------------------------------
# Entry
# -------------------------------------------------
if __name__ == "__main__":
    # asyncio.run(run_scenario())
    await run_scenario()



--- Turn 1: Capturing Information ---
Turn 1 completed & added to memory.

--- Turn 2: Recalling From Memory Tool ---


Event from an unknown agent: InfoCaptureAgent, event id: 9a75364b-4ed7-4f1f-b414-edc24fc72c13
Event from an unknown agent: InfoCaptureAgent, event id: 703ea5ee-0e3a-4ce8-bdf0-8e279b943e34


Agent 2 Response: Your favorite project is Project Alpha.

--- Turn 3: Loading From SQLite Session History ---
Agent 3 Response: You told me your favorite project is Project Alpha.
